In [112]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://scheduling.rutgers.edu/scheduling/academic-calendar'
requests.get(url)

pages = requests.get(url)
pages.text

soup = BeautifulSoup(pages.text, 'lxml')

table1 = soup.find('table')

headers=[]
for i in table1.find_all('th'):
  title = i.text
  headers.append(title)

mydata = pd.DataFrame(columns = headers)
prevFirst = ""
firstMonths = ['september','october','november','december']
hold = []
for j in table1.find_all('tr')[1:]:
  row_data = j.find_all('td')
  row = [i.text for i in row_data]
  if len(row) == 3:
    row.insert(0,prevFirst)
  counter = 0
  for data in row:
    data = data.lower()
    data = data.strip()
    if '-' in data:
      data = data[0:data.index('-')]
    if '–' in data:
      data = data[0:data.index('–')]
    data = repr(data).replace(r'\n',' ')
    data = data.replace(r'\t\t\t','')
    if counter != 0 :
      check = True
      yearOne = headers[counter][0:4]
      yearTwo = headers[counter][5:9]
      for m in firstMonths:
        if m in data:
          check = False
          data = data+yearOne
      if check:
        data = data + yearTwo
    row[counter] = data
    counter += 1
  length = len(mydata)
  mydata.loc[length] = row
  prevFirst = row[0]

mydata = mydata.drop(3)
mydata = mydata.drop(2)
mydata = mydata.drop(6)
mydata = mydata.drop(15)

end = ['\'thanksgiving recess end\'','\'sunday, november 28\'2021','\'sunday, november 27\'2022','\'sunday, november 26\'2023']
mydata.loc[length]=end

mydata.to_csv('calendar.csv', index=False)
mydata2 = pd.read_csv('calendar.csv')
print (mydata2)

                               Event  \
0             'fall semester begins'   
1   'regular saturday classes start'   
2              'thanksgiving recess'   
3              'regular classes end'   
4                 'fall exams begin'   
5                   'fall exams end'   
6            'winter session begins'   
7              'winter session ends'   
8           'spring semester begins'   
9             'spring recess begins'   
10              'spring recess ends'   
11             'regular classes end'   
12              'spring exams begin'   
13                'spring exams end'   
14                         'rutgers'   
15           'summer session begins'   
16         'thanksgiving recess end'   

                                            2021–2022  \
0   'wednesday, september 1 (no class: mon. 9/6)'2021   
1                         'saturday, september 4'2021   
2                       'wednesday, november 24 '2021   
3                           'monday, december 13'20